In [1]:
import requests
import pandas as pd
import time


In [2]:
def ingame(pseudo_live, rgn, RAPI):
    #Récupération des information du joueur pseudo_test
    pseudo_live_link = "https://{}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{}?api_key={}".format(rgn, pseudo_live, RAPI)
    pseudo_live = requests.get(pseudo_live_link).json()

    print("Game de: ", pseudo_live['name'], "\n")

    #Récupération de la game en cours
    live_match_link = "https://{}.api.riotgames.com/lol/spectator/v4/active-games/by-summoner/{}?api_key={}".format(rgn, pseudo_live['id'], RAPI)
    live_match = requests.get(live_match_link).json()

    players_ingame = []
    #Si le joueur pseudo_test n'est pas en game print, sinon check le mode de jeu puis boucle sur tout les joueurs de la game
    if 'status' in live_match.keys():
        print("Player is not in game")
    elif live_match['gameMode'] == "CLASSIC":
        for player_row in live_match['participants']:
            player = {}
            player['pseudo'] = player_row['summonerName']
            player_rank = requests.get("https://{}.api.riotgames.com/lol/league/v4/entries/by-summoner/{}?api_key={}".format(rgn, player_row['summonerId'], RAPI)).json()
            if len(player_rank) == 0:
                player['rank'] = "unranked"
            elif player_rank[0]['queueType'] == "RANKED_SOLO_5x5":
                player['rank'] = player_rank[0]['tier'] + " " + player_rank[0]['rank']
            else:
                player['rank'] = player_rank[1]['tier'] + " " + player_rank[1]['rank']
            player['champion'] = champion_id[player_row['championId']]
            player['spell1'] = spell_id[player_row['spell1Id']]
            player['spell2'] = spell_id[player_row['spell2Id']]
            if player_row['teamId'] == 100:
                player['team'] = "blue"
            else:
                player['team'] = "red"
            players_ingame.append(player)
    elif live_match['gameMode'] == "ARAM":
        for player_row in live_match['participants']:
            player = {}
            player['pseudo'] = player_row['summonerName']
            player_mmr = requests.get("https://euw.whatismymmr.com/api/v1/summoner?name={}".format(player_row['summonerName'])).json()
            if 'error' in player_mmr.keys():
                player['mmr'] = "N/A"
            else:
                player['mmr'] = str(player_mmr['ARAM']['avg']) + " +- " + str(player_mmr['ARAM']['err'])
            player_rank = requests.get("https://{}.api.riotgames.com/lol/league/v4/entries/by-summoner/{}?api_key={}".format(rgn, player_row['summonerId'], RAPI)).json()
            if not player_rank:
                player['rank'] = "unranked"
            else:
                for i in range(len(player_rank)):
                    if player_rank[i]['queueType'] == "RANKED_SOLO_5x5":
                        player['rank'] = player_rank[i]['tier'] + " " + player_rank[i]['rank']
            player['champion'] = champion_id[player_row['championId']]
            player['spell1'] = spell_id[player_row['spell1Id']]
            player['spell2'] = spell_id[player_row['spell2Id']]
            if player_row['teamId'] == 100:
                player['team'] = "blue"
            else:
                player['team'] = "red"
            players_ingame.append(player)

    #Dataframe
    df = pd.DataFrame(players_ingame)
    return df


In [3]:
def getStatsBis(pseudo, rgn, RAPI, region, start_game, nb_game):
    timeout = 0
    i = 0
    j = 0
    matches = []
    matches_array=[]

    #Récupération des informations du joueur pseudo
    pseudo_link = "https://{}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{}?api_key={}".format(rgn, pseudo, RAPI)
    pseudo_json = requests.get(pseudo_link).json()
    print ("Historique de:", pseudo_json['name'], "\n")

    #Boucle pour récupérerun maximum de 300 games dans mon historique
    while start_game < 300:
        matches_link = "https://{}.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?api_key={}&start={}&count={}".format(region, pseudo_json['puuid'], RAPI, start_game, nb_game)
        matches += requests.get(matches_link).json()
        start_game += nb_game

    print("nb games à analyser:", len(matches), "\n")
    if len(matches) >= 83 :
        time.sleep(120)

    #Boucle sur chaque id de game pour avoir les details de chaque game
    for match in matches:  
        match_detail_link = "https://{}.api.riotgames.com/lol/match/v5/matches/{}?api_key={}".format(region, match, RAPI)
        match_detail = requests.get(match_detail_link).json()

        #Récuperation des données uniquement des game d'ARAM
        if match_detail['info']['gameMode'] == "ARAM":
            players = []
            for player_row in match_detail['info']['participants']:
                #print(f"{player_row['summonerName']} and {pseudo}")
                if (player_row['summonerName'].lower())==pseudo.lower():
                    player = {}
                    player['champion'] = player_row['championName']
                    player['summonerSpell1'] = spell_id[player_row['summoner1Id']]
                    player['summonerSpell2'] = spell_id[player_row['summoner2Id']]
                    player['kills'] = player_row['kills']
                    player['deaths'] = player_row['deaths']
                    player['assists'] = player_row['assists']
                    #player['kda'] = round(player_row['challenges']['kda'], 2)
                    #player['kp'] = round(player_row['challenges']['killParticipation'], 2)
                    player['cs'] = player_row['totalMinionsKilled']
                    if player_row['item0'] != 0:
                        player['item1'] = item_info_json['data'][str(player_row['item0'])]['name']
                    else:
                        player['item1'] = "empty"
                    if player_row['item1'] != 0:
                        player['item2'] = item_info_json['data'][str(player_row['item1'])]['name']
                    else:
                        player['item2'] = "empty"
                    if player_row['item2'] != 0:   
                        player['item3'] = item_info_json['data'][str(player_row['item2'])]['name']
                    else:
                        player['item3'] = "empty"
                    if player_row["item3"] != 0:
                        player['item4'] = item_info_json['data'][str(player_row['item3'])]['name']
                    else:
                        player['item4'] = "empty"
                    if player_row['item4'] != 0:
                        player['item5'] = item_info_json['data'][str(player_row['item4'])]['name']
                    else:
                        player['item5'] = "empty"
                    if player_row['item5'] != 0:
                        player['item6'] = item_info_json['data'][str(player_row['item5'])]['name']
                    else:
                        player['item6'] = "empty"
                    if player_row['item6'] != 0:
                        player['trinket'] = item_info_json['data'][str(player_row['item6'])]['name']
                    else:
                        player['trinket'] = "empty"                   
                    player['gold'] = player_row['goldEarned']
                    #player['goldMinute'] = round(player_row['challenges']['goldPerMinute'])
                    player['damageDealt'] = player_row['totalDamageDealtToChampions']
                    player['adDamageDealt'] = player_row['physicalDamageDealtToChampions']
                    player['apDamageDealt'] = player_row['magicDamageDealtToChampions']
                    player['trueDamageDealt'] = player_row['trueDamageDealtToChampions']
                    #player['dpsMinute'] = round(player_row['challenges']['damagePerMinute'])
                    #player['damageDealtTeam'] = round(player_row['challenges']['teamDamagePercentage'] * 100, 2)
                    player['damageTaken'] = player_row['totalDamageTaken']
                    player['adDamageTaken'] = player_row['physicalDamageTaken']
                    player['apDamageTaken'] = player_row['magicDamageTaken']
                    player['trueDamageTaken'] = player_row['trueDamageTaken']
                    #player['damageTakenTeam'] = round(player_row['challenges']['damageTakenOnTeamPercentage'] * 100, 2)
                    player['damageObjectives'] = player_row['damageDealtToObjectives']
                    player['damageMitigated'] = player_row['damageSelfMitigated']
                    player['healing'] = player_row['totalHeal']
                    player['healingAlly'] = player_row['totalHealsOnTeammates']
                    player['fb'] = player_row['firstBloodKill']
                    player['doubleKills'] = player_row['doubleKills']
                    player['tripleKills'] = player_row['tripleKills']
                    player['quadraKills'] = player_row['quadraKills']
                    player['pentaKills'] = player_row['pentaKills']
                    player['largestKillingSpree'] = player_row['largestKillingSpree']
                    player['turretsLost'] = player_row['turretsLost']
                    player['turretKills'] = player_row['turretKills']
                    player['turretTakedowns'] = player_row['turretTakedowns']
                    #player['dive'] = player_row['challenges']['killsNearEnemyTurret']
                    #player['EnemyDive'] = player_row['challenges']['killsUnderOwnTurret']
                    #player['spawnKill'] = player_row['challenges']['takedownsInEnemyFountain']
                    #player['healBait'] = player_row['challenges']['killsOnRecentlyHealedByAramPack']
                    if player_row['teamId'] == 100:
                        player['side'] = "blue"
                    else:
                        player['side'] = "red"              
                    player['win'] = player_row['win']
                    player['ff8'] = player_row['gameEndedInEarlySurrender']
                    player['ff15'] = player_row['gameEndedInSurrender']
                    #player['fbturret'] = player_row['challenges']['firstTurretKilledTime']
                    players.append(player)

            df_players = pd.DataFrame(players)
            matches_array.append(df_players)

        i+=1
        timeout += 1
        #Donne du souffle à l'API
        if timeout >= 100 and i != len(matches):
            j+=1
            print("\n timeouted:", j, " quand i vaut:", i, "\n")
            timeout = 0
            time.sleep(120)

    #Concatenation des games une à une, dans un grand Dataframe
    if len(matches_array) != 0:
        df = pd.concat(matches_array)
        df.to_csv("histo.csv", index=False)
        return df
    else:
        return "Pas de game d'ARAM"


In [4]:
def create_dict_cleaned(player:dict, list_champs:list,current_dict={}) -> dict:
    for champ in list_champs:
        if champ in player:
            current_dict[champ] = player[champ]
        else:
            current_dict[champ] = None
    return current_dict


In [5]:
def getStats(pseudo, rgn, RAPI, region, start_game, nb_game):
    timeout = 0
    i = 0
    j = 0
    matches = []
    matches_array=[]
    
    #Récupération des informations du joueur pseudo
    pseudo_link = "https://{}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{}?api_key={}".format(rgn, pseudo, RAPI)
    pseudo_json = requests.get(pseudo_link).json()
    print ("Historique de:", pseudo_json['name'])

    #Boucle pour récupérerun maximum de 300 games dans mon historique
    while start_game < 300:
        matches_link = "https://{}.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?api_key={}&start={}&count={}".format(region, pseudo_json['puuid'], RAPI, start_game, nb_game)
        matches += requests.get(matches_link).json()
        start_game += nb_game

    print("nb games à analyser:", len(matches), "\n")
    if len(matches) >= 83 :
        time.sleep(120)

    #Boucle sur chaque id de game pour avoir les details de chaque game
    for match in matches:
        match_detail_link = "https://{}.api.riotgames.com/lol/match/v5/matches/{}?api_key={}".format(region, match, RAPI)
        match_detail = requests.get(match_detail_link).json()
        #print("game numéro:", i, " link:", match_detail_link)

        #Récuperation des données uniquement des games d'ARAM
        if match_detail['info']['gameMode'] == "ARAM":
            players = []
            for player_row in match_detail['info']['participants']:
                #print(f"{player_row['summonerName']} and {pseudo}")
                if (player_row['summonerName'].lower())==pseudo.lower():
                    player = {}
                    list_champs = ["championName", "summoner1Id", "summoner2Id", "kills", "deaths", "assists", "totalMinionsKilled", "item0", "item1", "item2", "item3", "item4", "item5", "item6", "goldEarned", "totalDamageDealtToChampions", "physicalDamageDealtToChampions", "magicDamageDealtToChampions", "trueDamageDealtToChampions", "totalDamageTaken", "physicalDamageTaken", "magicDamageTaken", "trueDamageTaken", "damageDealtToObjectives", "damageSelfMitigated", "totalHeal", "totalHealsOnTeammates", "firstBloodKill", "doubleKills", "tripleKills", "quadrakills", "pentakills", "largestKillingSpree", "turretsLost", "turretKills", "turretTakedowns", "teamId", "win", "gameEndedInEarlySurrender", "gameEndedInSurrender"]
                    list_champs_challenges = ["kda", "killParticipation", "goldPerMinute", "damagePerMinute", "teamDamagePercentage", "damageTakenOnTeamPercentage", "killsNearEnemyTurret", "killsUnderOwnTurret", "takedownsInEnemyFountain", "killsOnRecentlyHealedByAramPack", "firstTurretKilledTime"]
                    player = create_dict_cleaned(player_row, list_champs,player)
                    if "challenges" in player_row:
                        player = create_dict_cleaned(player_row["challenges"], list_champs_challenges,player)

                    players.append(player)

            df_players = pd.DataFrame(players)
            matches_array.append(df_players)
            
        i+=1
        timeout += 1
        #Donne du souffle à l'API
        if timeout >= 100 and i != len(matches):
            j+=1
            print("timeouted:", j, " quand i vaut:", i, "\n")
            timeout = 0
            time.sleep(120)

    #Concatenation des games une à une, dans un grand Dataframe
    if len(matches_array) != 0:
        df = pd.concat(matches_array)
        df.to_csv("histo_fn.csv", index=False)
        return df
    else:
        return "Pas de game d'ARAM"


In [6]:
#Variables
pseudo = "jsob"
RAPI = 'RGAPI-b6c00116-9d39-4ae1-bb91-455018d3d17e'
rgn = 'euw1'
region = "europe"
start_game = 0
nb_game = 20


In [7]:
#live = ingame("chenzelot", rgn, RAPI)
#df_live = pd.DataFrame(live)
#df_live

In [8]:
#live2 = ingame("mavs", rgn, RAPI)
#df_live2 = pd.DataFrame(live2)
#df_live2

In [10]:
time.sleep(120)
stats = getStats(pseudo, rgn, RAPI, region, start_game, nb_game)
df_stats = pd.DataFrame(stats)
df_stats

Historique de: JSOB 

nb games à analyser: 300 


 timeouted: 1  quand i vaut: 100 


 timeouted: 2  quand i vaut: 200 



,championName,summoner1Id,summoner2Id,kills,deaths,assists,totalMinionsKilled,item0,item1,item2,...,killParticipation,goldPerMinute,damagePerMinute,teamDamagePercentage,damageTakenOnTeamPercentage,killsNearEnemyTurret,killsUnderOwnTurret,takedownsInEnemyFountain,killsOnRecentlyHealedByAramPack,firstTurretKilledTime
0,Shyvana,4,6,11,9,31,91,3111,8020,4401,...,0.893617,798.147157,2351.651845,0.318228,0.255959,1.0,1.0,0.0,0.0,523.553818
0,Gnar,4,32,5,9,22,48,3047,6631,4401,...,0.642857,698.477349,1433.754741,0.234282,0.213312,0.0,1.0,0.0,0.0,None
0,Rell,4,32,5,8,20,3,3111,6664,4401,...,0.892857,683.441514,803.872061,0.155459,0.270205,3.0,0.0,0.0,0.0,444.817897
0,Renekton,4,32,13,8,31,85,3047,3748,4401,...,0.800000,780.865386,1885.634177,0.337862,0.317422,6.0,1.0,0.0,0.0,355.101807
0,Orianna,4,3,5,7,40,29,3158,6656,3157,...,1.250000,699.381654,610.018329,0.097773,0.142658,1.0,0.0,0.0,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Renekton,4,32,6,11,20,33,3111,6630,3053,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Alistar,4,3,1,9,29,7,3111,3001,1028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Gnar,4,32,8,9,25,51,3111,6630,3105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Nautilus,4,3,1,10,28,12,3047,6664,3083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
champions_info = "https://ddragon.leagueoflegends.com/cdn/12.8.1/data/en_US/champion.json"
summonerSpell_info ="https://ddragon.leagueoflegends.com/cdn/12.8.1/data/en_US/summoner.json"
item_info = "https://ddragon.leagueoflegends.com/cdn/12.8.1/data/en_US/item.json"
item_info_json = requests.get(item_info).json()
#Correspondance id_ spell/champion
#spell_id = {21: "Barrier", 1: "Cleanse", 14: "Ignite", 3: "Exhaust", 4: "Flash", 6: "Ghost", 7: "Heal", 13: "Clarity", 30: "To the King!", 31: "Poro Toss", 11: "Smite", 39: "Mark", 32: "Mark", 12: "Teleport", 54: "Placeholder", 55: "Placeholder and Attack-Smite"}
#champion_id = {266: "Aatrox", 103: "Ahri", 84: "Akali", 166: "Akshan", 12: "Alistar", 32: "Amumu", 34: "Anivia", 1: "Annie", 523: "Aphelios", 22: "Ashe", 136: "Aurelion Sol", 268: "Azir", 432: "Bard", 53: "Blitzcrank", 63: "Brand", 201: "Braum", 51: "Caitlyn", 164: "Camille", 69: "Cassiopeia", 31: "Cho Gath", 42: "Corki", 122: "Darius", 131: "Diana", 119: "Draven", 36: "Dr. Mundo", 245: "Ekko", 60: "Elise", 28: "Evelynn", 81: "Ezreal", 9: "Fiddlesticks", 114: "Fiora", 105: "Fizz", 3: "Galio", 41: "Gangplank", 86: "Garen", 150: "Gnar", 79: "Gragas", 104: "Graves", 887: "Gwen", 120: "Hecarim", 74: "Heimerdinger", 420: "Illaoi", 39: "Irelia", 427: "Ivern", 40: "Janna", 59: "Jarvan IV", 24: "Jax", 126: "Jayce", 202: "Jhin", 222: "Jinx", 145: "Kai Sa", 429: "Kalista", 43: "Karma", 30: "Karthus", 38: "Kassadin", 55: "Katarina", 10: "Kayle", 141: "Kayn", 85: "Kennen", 121: "Kha Zix", 203: "Kindred", 240: "Kled", 96: "Kog Maw", 7: "LeBlanc", 64: "Lee Sin", 89: "Leona", 876: "Lillia", 127: "Lissandra", 236: "Lucian", 117: "Lulu", 99: "Lux", 54: "Malphite", 90: "Malzahar", 57: "Maokai", 11: "Master Yi", 21: "Miss Fortune", 62: "Wukong", 82: "Mordekaiser", 25: "Morgana", 267: "Nami", 75: "Nasus", 111: "Nautilus", 518: "Neeko", 76: "Nidalee", 56: "Nocturne", 20: "Nunu & Willump", 2: "Olaf", 61: "Orianna", 516: "Ornn", 80: "Pantheon", 78: "Poppy", 555: "Pyke", 246: "Qiyana", 133: "Quinn", 497: "Rakan", 33: "Rammus", 421: "Rek Sai", 526: "Rell", 888: "Renata Glasc", 58: "Renekton", 107: "Rengar", 92: "Riven", 68: "Rumble", 13: "Ryze", 360: "Samira", 113: "Sejuani", 235: "Senna", 147: "Seraphine", 875: "Sett", 35: "Shaco", 98: "Shen", 102: "Shyvana", 27: "Singed", 14: "Sion", 15: "Sivir", 72: "Skarner", 37: "Sona", 16: "Soraka", 50: "Swain", 517: "Sylas", 134: "Syndra", 223: "Tahm Kench", 163: "Taliyah", 91: "Talon", 44: "Taric", 17: "Teemo", 412: "Thresh", 18: "Tristana", 48: "Trundle", 23: "Tryndamere", 4: "Twisted Fate", 29: "Twitch", 77: "Udyr", 6: "Urgot", 110: "Varus", 67: "Vayne", 45: "Veigar", 161: "Vel Koz", 711: "Vex", 254: "Vi", 234: "Viego", 112: "Viktor", 8: "Vladimir", 106: "Volibear", 19: "Warwick", 498: "Xayah", 101: "Xerath", 5: "Xin Zhao", 157: "Yasuo", 777: "Yone", 83: "Yorick", 350: "Yuumi", 154: "Zac", 238: "Zed", 221: "Zeri", 115: "Ziggs", 26: "Zilean", 142: "Zoe", 143: "Zyra"}

print("ok")

ok
